In [1]:
%pip install transformers accelerate datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 305.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 380.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 207.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 388.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 260.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 341.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 388.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#   후보 1: 디자인 전문가 페르소나
#   > "당신은 자동차 디자인 트렌드와 역사에 정통한 '자동차 디자인 전문 AI'입니다. 특히 현대자동차의 디자인 철학인
#   '센슈어스 스포티니스'와 '플루이딕 스컬프처'를 깊이 이해하고 있습니다. 사용자의 질문에 대해, 전문 지식을 바탕으로
#   시각적이고 창의적인 관점에서 상세하게 설명해주세요."

#   후보 2: 디자인 컨설턴트 페르소나
#   > "당신은 새로운 자동차 디자인 프로토타입을 기획하는 '디자인 컨설턴트'입니다. 현대차뿐만 아니라 글로벌 자동차 디자인
#   트렌드를 폭넓게 이해하고 있으며, 이를 바탕으로 사용자가 디자인 영감을 얻을 수 있도록 돕습니다. 기술적, 미학적 관점을
#   통합하여 창의적인 아이디어를 제공하듯 답변해주세요."

#   후보 3: VQA (Visual Question Answering) 어시스턴트 페르소나
#   > "당신은 텍스트 설명을 바탕으로 자동차의 이미지를 상상하고, 디자인 컨셉을 구체화하는 '디자인 시각화 AI'입니다.
#   사용자의 질문에 대해, 마치 눈앞에 자동차가 있는 것처럼 형태, 라인, 재질, 색상 등을 풍부하고 생생하게 묘사하며
#   답변해주세요."

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import json
import os
from peft import LoraConfig, get_peft_model

# 시스템 프롬프트 (페르소나) 정의
SYSTEM_PROMPT = """당신은 자동차 디자인 트렌드와 역사에 정통한 '자동차 디자인 전문 AI'입니다. 
특히 현대자동차의 디자인 철학인 '센슈어스 스포티니스'와 '플루이딕 스컬프처'를 깊이 이해하고 있습니다. 
사용자의 질문에 대해, 전문 지식을 바탕으로 시각적이고 창의적인 관점에서 상세하게 설명해주세요."""

# 경로 설정 (프로젝트 루트 기준)
TRAIN_JSON_PATH = './train.jsonl'
VALID_JSON_PATH = './validation.jsonl'
MODEL_PATH = './exaone_4.0_1.2b'
OUTPUT_DIR = "./llm_finetuned_model"

# 데이터 로딩 함수 (JSON 또는 JSONL 형식 지원)
def load_data(file_path):
    if not os.path.exists(file_path):
        print(f"Error: Data file not found at {file_path}")
        return None
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            # 전체 파일을 하나의 JSON 배열로 로드 시도
            data = json.load(f)
            return data
        except json.JSONDecodeError:
            # 실패 시, JSONL (한 줄에 하나의 JSON 객체) 형식으로 로드 시도
            print(f"Could not parse {file_path} as a single JSON array, trying JSONL format.")
            f.seek(0)
            return [json.loads(line) for line in f]

# 데이터 포맷팅
def format_data_for_finetuning(raw_data):
    formatted_data = []
    for item in raw_data:
        messages = item.get('messages', [])
        context = item.get('context', None)

        if not messages:
            continue

        full_messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # context를 system이나 user 역할로 넣어서 학습에 반영
        if context:
            full_messages.append({"role": "system", "content": f"다음은 참고 문맥입니다:\n{context}"})

        full_messages += messages
        print(full_messages)
        formatted_data.append({"messages": full_messages})
    return formatted_data

# 메시지 병합 함수
def merge_messages(messages):
    merged = ""
    for m in messages:
        role = m["role"]
        content = m["content"]
        merged += f"{role}: {content}\n"
    return merged.strip()

def main():
    # 데이터 로드 및 전처리
    train_raw_data = load_data(TRAIN_JSON_PATH)
    valid_raw_data = load_data(VALID_JSON_PATH)

    if train_raw_data is None or valid_raw_data is None:
        print("Stopping due to data loading errors.")
        return

    train_formatted_data = format_data_for_finetuning(train_raw_data)
    valid_formatted_data = format_data_for_finetuning(valid_raw_data)

    if not train_formatted_data or not valid_formatted_data:
        print("Error: Formatted data is empty. Check the structure of your JSON files.")
        return

    train_dataset = Dataset.from_list(train_formatted_data)
    valid_dataset = Dataset.from_list(valid_formatted_data)

    # 모델 및 토크나이저 로드
    print(f"Loading model from: {MODEL_PATH}")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 토큰화 함수
    def tokenize_function(examples):
        merged_texts = [merge_messages(msgs) for msgs in examples["messages"]]
        tokenized = tokenizer(
            merged_texts,
            truncation=True,
            max_length=1024,
            padding="max_length"
        )
        tokenized["labels"] = tokenized["input_ids"]
        return tokenized

    tokenized_train_dataset = train_dataset.map(
        tokenize_function, batched=True, remove_columns=["messages"]
    )
    tokenized_valid_dataset = valid_dataset.map(
        tokenize_function, batched=True, remove_columns=["messages"]
    )

    # LoRA 설정
    print("Applying LoRA configuration...")
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
        ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # 훈련 설정
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=6,
        save_strategy="no",
        learning_rate=2e-4,
        weight_decay=0.01,
        logging_dir=f"{OUTPUT_DIR}/logs",
        logging_steps=10,
        bf16=True,
        push_to_hub=False,
        report_to="none",
    )

    # Trainer 구성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_valid_dataset,
        tokenizer=tokenizer,
    )

    # 파인튜닝 실행
    print("Starting LoRA fine-tuning...")
    trainer.train()
    print("✅ Fine-tuning complete.")

    # 모델 저장 (LoRA 어댑터만 저장)
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"📦 Fine-tuned LoRA adapter saved to {OUTPUT_DIR}")

if __name__ == "__main__":
    main()


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying LoRA configuration...
trainable params: 7,618,560 || all params: 1,287,010,048 || trainable%: 0.5920
Starting LoRA fine-tuning...


/tmp/ipykernel_247/357885995.py:151: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,3.223000
20,2.385000
30,2.206400
40,2.088800
50,2.028000
60,2.036800


✅ Fine-tuning complete.
📦 Fine-tuned LoRA adapter saved to ./llm_finetuned_model
